In [2]:
import pandas as pd
import numpy as np
import os

## 2017 Lezhin Comics Data Challenge

레진코믹스의 마케팅팀은 데이터 분석가(Data Analyst)의 분석 결과를 바탕으로 사용 플랫폼별 마케팅 예산을 재조정(rebalancing)할 수 있다. 운영팀과 기획팀은 분석 결과를 반영하여 매출이 잘 나오는 작품으로 트래픽을 몰아줄 수 있다. 재무팀은 분석 결과를 통해 앞으로의 매출을 예상할 수 있을 것이다.

반면 데이터 분석가가 정확한 분석 결과를 전달하지 못할 경우, 마케팅팀은 마케팅 예산을 재조정하는데 실패함으로써 회사의 매출을 감소시킬 수 있다. 또한 운영팀과 기획팀은 매출이 잘 나오지 않는 컨텐츠에 많은 시간과 비용을 투입함으로써 회사에 손해를 안길 것고, 재무팀은 회사의 앞으로의 재무상황을 잘못 예측함으로써 큰 위기에 빠질 수 있다.

그러므로 데이터를 정확하게 분석하는 것은 무엇보다도 중요하다고 할 수 있다. 

**Dataset**

* File format: TSV
* Size: 228M
* Number of samples: 650,965
* Number of features: 167

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 6 : 목록 진입시점 방문 총 세션 수 (범위별로 부여된 순차 ID)
  * 7 : 작품을 나타내는 해쉬
  * 8-10 : 개인정보
  * 11-110 : 주요 작품 구매 여부
  * 111 : 작품 태그 정보
  * 112 : 구매할 때 필요한 코인
  * 113 : 완결 여부
  * 114-123 : 스케쥴 정보
  * 124-141 : 장르 정보
  * 142 : 해당 작품의 마지막 에피소드 발행 시점 (범위별로 부여된 순차 ID)
  * 143 : 단행본 여부
  * 144 : 작품 발행 시점 (범위별로 부여된 순차 ID)
  * 145 : 총 발행 에피소드 수 (범위별로 부여된 순차 ID)
  * 146-151 : 작품 태그 정보
  * 152-167 : 유저의 성향 정보 (과거에 구매를 했을 때만 기록)

## Load Dataset

In [4]:
data = pd.read_csv("lezhin_dataset_v2_training.tsv", sep="\t", header=None)

print(data.shape)
data.head()

(650965, 167)


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,1,1,001C9D9B,8337B6FB,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2580,0.0030,NaN,NaN
1,0,0,0,0,1,1,001C9D9B,90D8AB70,1,NaN,...,NaN,0.5000,NaN,NaN,NaN,NaN,0.5000,NaN,NaN,NaN
2,0,0,0,0,1,1,001C9D9B,ABC21E80,1,NaN,...,NaN,0.0187,0.0047,NaN,NaN,NaN,0.0287,0.0055,NaN,NaN
3,0,0,0,0,1,1,001C9D9B,C17967D1,0,69EF2C8F,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0186,NaN,NaN,NaN
4,0,0,0,0,1,1,002B4BDE,AF145784,0,NaN,...,NaN,0.0207,NaN,NaN,NaN,NaN,0.2805,0.0692,NaN,NaN


### 1. 사용 플랫폼 분석

레진코믹스는 기본적으로 4개의 플랫폼으로 서비스의 홍보를 합니다. (사용 플랫폼 A ~ D) 레진코믹스의 고객은 이 4개의 플랫폼 중 하나로 유입되며, 레진코믹스 서비스를 둘러보다가 특정 웹툰을 구매합니다. (또는 아무것도 구매하지 않고 서비스를 빠져나갑니다)

그러므로 마케팅 팀 입장에서는 사용 플랫폼 A ~ D의 광고 효율을 주기적으로 체크할 수 있어야 합니다. 가령 사용 플랫폼 A보다 D가 더 효율적이라는 분석 결과가 나오면, 마케팅 팀에서는 사용 플랫폼 D에 투입하는 마케팅 예산을 늘리고, 반대로 사용 플랫폼 A에 투입하는 예산을 줄여 총 매출을 증가시킬 수 있습니다.

그러므로 사용 플랫폼별 구매 횟수와 매출(=코인)을 분석해보겠습니다. 다음의 컬럼이 필요합니다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 2 : 사용 플랫폼 A
  * 3 : 사용 플랫폼 B
  * 4 : 사용 플랫폼 C
  * 5 : 사용 플랫폼 D
  * 112 : 구매할 때 필요한 코인

In [5]:
platform = data[[0, 1, 2, 3, 4]].copy()
platform.columns = ["구매 여부", "사용 플랫폼 A", "사용 플랫폼 B",
                    "사용 플랫폼 C", "사용 플랫폼 D"]
platform["구매 코인"] = data[111]

print(platform.shape)
platform.head()

(650965, 6)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,구매 코인
0,0,0,0,0,1,4
1,0,0,0,0,1,4
2,0,0,0,0,1,4
3,0,0,0,0,1,4
4,0,0,0,0,1,3


**1. 구매한 사용자 / 구매하지 않은 사용자의 인원 수를 찾기**

In [6]:
platform["구매 여부"].value_counts()

0    429172
1    221793
Name: 구매 여부, dtype: int64

구매 확률은 약 34%로 구매하지 않은 사용자가 구매한 사용자보다 많다.

In [36]:
platform.pivot_table(values = ['사용 플랫폼 A', '사용 플랫폼 B', '사용 플랫폼 C', '사용 플랫폼 D'],
                     index='구매 여부', aggfunc=sum)

,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
구매 여부,,,,
0,74685,149623,63049,141815
1,15108,42731,45023,118931


In [13]:
print(platform.groupby('구매 여부').sum().sum().drop('구매 코인'))
print(platform.groupby('구매 여부').sum().sum().drop('구매 코인').sum())

사용 플랫폼 A     89793
사용 플랫폼 B    192354
사용 플랫폼 C    108072
사용 플랫폼 D    260746
dtype: int64
650965


In [14]:
print(platform.shape)

(650965, 6)


사용 플랫폼 A ~ D 중 두 개 이상의 플랫폼에서 동시에 방문하는 경우는 없다. 즉, 사용자는 언제나 사용 플랫폼 A, B, C, D 중 한 곳에서만 유입된다.

그러므로 사용 플랫폼 A ~ D를 따로 나누지 않고, **사용 플랫폼**이라는 하나의 컬럼으로 합치자.

In [15]:
platform.loc[platform["사용 플랫폼 A"] == 1, "사용 플랫폼"] = "A"
platform.loc[platform["사용 플랫폼 B"] == 1, "사용 플랫폼"] = "B"
platform.loc[platform["사용 플랫폼 C"] == 1, "사용 플랫폼"] = "C"
platform.loc[platform["사용 플랫폼 D"] == 1, "사용 플랫폼"] = "D"

print(platform.shape)

platform[["사용 플랫폼", "사용 플랫폼 A", "사용 플랫폼 B", "사용 플랫폼 C", "사용 플랫폼 D"]].head()

(650965, 7)


,사용 플랫폼,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D
0,D,0,0,0,1
1,D,0,0,0,1
2,D,0,0,0,1
3,D,0,0,0,1
4,D,0,0,0,1


## 매출 분석

**3. 사용 플랫폼별 구매한 사용자의 총 인원 수와 평균 구매율을 찾기**

먼저, 어떤 플랫폼에서 사람들이 많이 유입되는지를 알아보자. (해당 플랫폼이 레진코믹스의 매출을 담당하는 핵심 채널이라고 판단할 수 있습니다)

또한 총 구매 인원 수와는 별개로, 각 사용 플랫폼에서 방문한 고객이 작품을 구매할 확률을 계산할 필요도 있다. 가령 사용 플랫폼 D의 총 구매 인원이 A의 두 배라고 하더라도, 마케팅팀에서 사용 플랫폼 D에 지출한 마케팅 예산이 A의 세 배였다면, 실질적으로 사용 플랫폼 D는 A에 비해 좋은 마케팅 채널이 아니라고 판단할 수 있다.

이 경우 마케팅팀은 사용 플랫폼 D에 책정한 마케팅 예산을 A로 옮기는 재조정(rebalancing) 작업을 거침으로써 레진코믹스의 매출을 늘릴 수 있을 것이다. 그러므로 사용 플랫폼별 누적 구매 인원(sum)은 물론, 구매 확률(mean)을 구할 필요가 있다.

In [18]:
sales = pd.pivot_table(platform,
               index='사용 플랫폼',
               values="구매 여부",
               aggfunc=[np.sum, np.mean]) 
sales

,sum,mean
,구매 여부,구매 여부
사용 플랫폼,,
A,15108,0.168254
B,42731,0.222148
C,45023,0.416602
D,118931,0.456118


In [19]:
sales.columns

MultiIndex(levels=[['sum', 'mean'], ['구매 여부']],
           labels=[[0, 1], [0, 0]])

In [20]:
sales.sort_values(by=("sum","구매 여부"), ascending=False)

,sum,mean
,구매 여부,구매 여부
사용 플랫폼,,
D,118931,0.456118
C,45023,0.416602
B,42731,0.222148
A,15108,0.168254


누적 구매량 관점에서 사용 플랫폼은 D > C > B > A 순으로 높다는 사실을 알 수 있다. 또한 구매 확률을 기준으로 해도 D > C > B > A 순으로 좋은 편인데, 눈여겨 볼 점은 B와 C는 누적 구매 인원은 비슷하지만 B가 C보다 구매 확률이 더 높다는 것이다.

마케팅팀에서 집행한 사용 플랫폼별 마케팅 예산을 비교해봐야 정확히 알 수 있지만, 이 분석 결과로는 사용 플랫폼 B가 예상보다 효율이 좋지 않다고 판단할 수 있다.(비슷하게 사용 플랫폼 A도 효율이 좋지 않다.) 이 가설이 맞다면 마케팅팀은 사용 플랫폼 B에 투입하는 광고 예산을 줄인 뒤, 이 예산을 사용 플랫폼 D나 C에 집행하는 것이 올바른 판단이다.

**4. 사용 플랫폼별 구매한 코인의 총 개수를 찾기**

레진코믹스의 구매 시스템을 파악하기 위해서는 '코인'이라는 개념을 이해하는 것이 중요합니다. 레진코믹스에서 웹툰을 보기 위해서는 1) 먼저 코인을 구매한 뒤, 2) 구매한 코인을 사용하여 웹툰을 결제합니다.

여기서 주의할 점은, 웹툰마다 구매하는데 필요한 코인이 다르다는 것이다. 그러므로 광고 효율을 정확하게 파악하기 위해서는, 누적 구매수가 아닌 누적 코인으로 계산하는 것이 더 정확하다.

이 사실을 바탕으로 사용 플랫폼별 광고 효율을 다시 한 번 계산해보자. 이번에는 사용 플랫폼별 누적 구매수와, 누적 구매 코인 수를 동시에 계산해보자.

In [21]:
# 먼저 전체 사용자에서 구매한 사용자만 색인합니다.
platform_purchased = platform[platform["구매 여부"] == 1]

print(platform_purchased.shape)
platform_purchased.head()

(221793, 7)


,구매 여부,사용 플랫폼 A,사용 플랫폼 B,사용 플랫폼 C,사용 플랫폼 D,구매 코인,사용 플랫폼
216289,1,0,0,0,1,4,D
216290,1,0,0,0,1,3,D
216291,1,0,0,0,1,3,D
216292,1,0,0,0,1,3,D
216293,1,0,0,0,1,3,D


In [22]:
table = pd.pivot_table(platform_purchased,
                       index='사용 플랫폼',
                       values=["구매 여부", "구매 코인"],
                       aggfunc='sum')
table

,구매 여부,구매 코인
사용 플랫폼,,
A,15108,35055
B,42731,102627
C,45023,119905
D,118931,314648


* 위 결과의 숫자들은 너무 크고 난해해서, 구체적으로 어떤 사용 플랫폼의 효율이 좋은지 판단하기 어렵다. 그러므로 분석 결과를 사람이 이해하기 쉬운 방식으로 변경해주자.

**5. 사용 플랫폼별 구매 코인과 구매 여부를 배수로 보여주기**

사용 플랫폼 A에 비하여 B, C, D가 얼마나 구매량이 높은지를 분석해보자. 가령 "구매 여부(%)"라는 컬럼에는 사용 플랫폼 A에 비해 사용 플랫폼 B, C, D가 누적 구매량이 몇 배 높은지를 보여주고, "구매 코인(%)" 컬럼에서는 사용 플랫폼 A에 비해 사용 플랫폼 B, C, D의 누적 구매 코인이 몇 배 높은지를 보여주자.

In [23]:
# 구매 여부를 A의 누적 구매량인 15108로 나누면 A대비 B, C, D에서 몇 배를 더 구매했는지 알 수 있다.
table["구매 여부(%)"] = table["구매 여부"] / 15108

# 비슷한 방식으로 구매 코인을 A의 누적 구매 코인 수인 35055로 나눈다.
table["구매 코인(%)"] = table["구매 코인"] / 35055

table

,구매 여부,구매 코인,구매 여부(%),구매 코인(%)
사용 플랫폼,,,,
A,15108,35055,1.000000,1.000000
B,42731,102627,2.828369,2.927599
C,45023,119905,2.980077,3.420482
D,118931,314648,7.872055,8.975838


* C의 경우 한 웹툰을 구매할 때 더 많은 코인을 쓴다는 것을 알 수 있다. (3.42 > 2.98) 
* D의 경우도 한 웹툰을 구매할 때 더 많은 코인을 쓴다는 사실을 알 수 있다. (8.97 > 7.87)

만일 마케팅팀이 현재까지 누적 코인이 아닌 누적 구매량으로 마케팅 예산을 배분하고 있었다면, 사용 플랫폼 C와 D에 마케팅 예산을 더 집중하는 쪽으로 재조정 한다면 마케팅 효율이 더 높아질 것으로 기대할 수 있다.

## 작품 분석

레진코믹스의 전체 사용자 트래픽를 어떤 웹툰에 몰아주는 것이 회사의 매출을 늘리는데 도움이 될까?

가령 레진코믹스의 메인 페이지에 웹툰 A, B, C라는 세 개의 웹툰을 번갈아서 보여주었을 때, A를 구매할 확률이 30%, B를 구매할 확률이 35%, C를 구매할 확률이 20%라면, 운영팀은 메인 페이지에 B 웹툰을 보여주는 것이 좋습니다. (A/B 테스팅)

하지만 메인 페이지에 보여줄 웹툰을 선택할 때에는, 구매할 확률 외에도 웹툰당 구매에 필요한 코인을 고려해야 한다. 만약 위의 예시에서 웹툰 A를 구매하는데 코인이 4개가 필요하고, 웹툰 B를 구매하는데 코인이 3개가 필요하다면, 운영팀은 메인 페이지에 웹툰 B가 아닌 A를 보여주는 선택을 하는 게 맞다.

여기에 추가로 고려할 건, 레진코믹스에 연재중인 웹툰은 너무 많기 때문에, 모든 웹툰을 위 예시처럼 메인 페이지에 번갈아서 올려보는 테스팅을 할 수 없다. 그러므로 과거의 구매 기록을 바탕으로 어떤 웹툰을 메인 페이지에 띄우는게 매출 상승에 도움이 되는지 사전에 판단할 필요가 있다.

그러므로 어떤 웹툰이 고객들에게 인기가 많고, 많은 매출을 달성했는지 (or 달성할 가능성이 있는지) 분석해보자. 

다음의 컬럼이 필요하다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 112 : 구매할 때 필요한 코인 (이하 구매 코인)

In [24]:
comics = data[[0, 6, 111]].copy()

comics.columns = ["구매 여부", "작품 번호", "구매 코인"]

print(comics.shape)
comics.head()

(650965, 3)


,구매 여부,작품 번호,구매 코인
0,0,001C9D9B,4
1,0,001C9D9B,4
2,0,001C9D9B,4
3,0,001C9D9B,4
4,0,002B4BDE,3


**6. 총 작품의 개수 구하기**

만일 레진코믹스에 연재중인 웹툰이 아주 많지 않다면, 과거 구매 기록을 분석할 필요하지 않아도 된다. (모든 웹툰을 메인페이지에 일일이 띄워서 구매 확률을 계산하면 충분하다.)

하지만 연재중인 웹툰이 아주 많다면 이 웹툰을 일일이 메인페이지에 띄울 수가 없다. 

그러므로 이를 확인하기 위해, 레진코믹스에 연재중인 웹툰이 총 개수를 구해보자.

In [25]:
# 작품 번호에서 중복된 값을 제거하기
comics_list = comics["작품 번호"].unique()

comics_list.shape

(3559,)

작품은 총 3,559 개로, 전부 메인페이지에 올려보기에는 너무 많다. 그러므로 선호도가 높거나 구매 코인이 많은 웹툰을 찾은 후, 해당 웹툰만 메인페이지에 올려보는게 좋을 것 같다.

**7. 사용자들이 가장 많이 구매한 작품 Top 10 찾기**

전체 웹툰을 레진코믹스의 사용자들이 가장 많이 구매한 웹툰 순으로 정렬해보자.

In [26]:
# 전체 데이터에서 구매한 데이터만 남기고 구매하지 않은 데이터는 버린다.
comics_purchased = comics[comics["구매 여부"] == 1]

print(comics_purchased.shape)
comics_purchased.head()

(221793, 3)


,구매 여부,작품 번호,구매 코인
216289,1,001C9D9B,4
216290,1,002B4BDE,3
216291,1,00A49090,3
216292,1,00A49090,3
216293,1,00A49090,3


In [27]:
# 각 제품당 총 구매 개수 구하기
comics_purchased_counts = comics_purchased["작품 번호"].value_counts()

comics_purchased_counts.head(10)

D619AC7D    5684
00A49090    5298
F0A27F22    3933
85803B60    3894
EC612BAF    3660
621BF66D    3318
BEE0E409    3298
46F57FFA    3103
6983846D    3035
707F7107    2623
Name: 작품 번호, dtype: int64

분석 결과 'D619AC7D', '00A49090', 'F0A27F22', '85803B60', 'EC612BAF', '621BF66D', 'BEE0E409', '46F57FFA', '6983846D', '707F7107' 웹툰 순으로 구매가 많이 일어났다는 사실을 알 수 있다.

**8. 구매 코인을 기준으로 가장 많은 매출을 달성한 작품 Top10 구하기**

7번 결과는 큰 도움이 되겠지만, 여기서도 마찬가지로 웹툰마다의 구매 코인이 다르다는 사실에 주의해야 한다. 즉, 구매 횟수가 아닌 구매 코인을 기준으로 가장 많이 구매한 웹툰 순으로 정렬할 필요가 있다.

In [28]:
comics_sales = comics_purchased.groupby("작품 번호")["구매 코인"].sum()
comics_sales = comics_sales.sort_values(ascending=False)

comics_sales.head(10)

작품 번호
D619AC7D    17052
00A49090    15894
F0A27F22    11799
85803B60    11682
EC612BAF    10980
621BF66D     9954
BEE0E409     9894
46F57FFA     9309
6983846D     9105
707F7107     7869
Name: 구매 코인, dtype: int64

분석 결과 'D619AC7D', '00A49090', 'F0A27F22', '85803B60', 'EC612BAF', '621BF66D', 'BEE0E409', '46F57FFA', '6983846D', '707F7107' 순으로 구매가 많이 일어난다는 사실을 알 수 있고, 다행히 7번 결과와 같다.

**9. 가장 구매 확률이 높은 작품 Top10 구하기**

구매 횟수와 구매 코인을 기준으로 정렬하는 것 모두 좋은 방법이다. 하지만 여기에는 한 가지 간과한 점이 있는데, 바로 특정 웹툰에 유입된 트래픽의 양이 다를 수가 있다는 사실이다.

가령 D 웹툰이 E 웹툰보다 코인 구매량이 1.7배가 많다고 하더라도, 애초부터 D 웹툰에 접근한 사용자가 E 웹툰에 접근한 사용자보다 2배 가까이 많았다면, 실제로는 E 웹툰이 더 사람들의 이목을 끄는 웹툰이라고 볼 수 있다. 즉, E가 acquisition cost가 더 낮은 제품이라는 것이다.

즉, 각 웹툰에 동일한 트래픽을 제공해준다는 가정 하에 E 웹툰을 메인 페이지에 띄우는 것이 D 웹툰을 띄우는 것 보다 더 많은 매출을 발생시킬 것으로 기대할 수 있다. 그렇기 때문에, 구매 횟수나 구매 코인이 아닌 구매 확률을 살펴볼 필요가 있다. 

사용자가 특정 웹툰에 접근했을 때, 해당 웹툰을 구매할 확률을 계산하자.

In [30]:
purchased_probability = pd.pivot_table(comics, index="작품 번호", values="구매 여부")   # mean값 계산됨
purchased_probability = purchased_probability.sort_values(by="구매 여부", ascending=False)

print(purchased_probability.shape)
purchased_probability.head()

(3559, 1)


,구매 여부
작품 번호,
82F2B308,1.0
87EEDD0C,1.0
F4C3DDD4,1.0
E1E8BE6B,1.0
F4BB2FE5,1.0


상위 10개의 구매 확률이 전부 1.0(100%)이라는 사실을 알 수 있다. 이는 실제로 구매 확률이 100%가 나올 거라기보다는, 웹툰을 구매한 횟수가 작아서 생기는 문제라고 볼 수 있다.

In [31]:
# 상위 10개의 웹툰의 작품 번호
top10 = purchased_probability.head(10).index

# 해당 10개의 작품 번호의 구매정보만을 따로 indexing
top_comics = comics[comics["작품 번호"].isin(top10)]

top_comics["작품 번호"].value_counts()

87B1AA23    3
BB835283    2
F342A1FD    2
82F2B308    2
F4BB2FE5    1
F4C3DDD4    1
87EEDD0C    1
39434E6C    1
E1E8BE6B    1
F401ED3F    1
Name: 작품 번호, dtype: int64

**10. 100회 이상 구매한 작품 중, 가장 구매 확률이 높은 작품 top 10을 찾기**

9번에서 발견한 문제(전부 1.0으로 나오는 문제)를 해결하기 위해, 이번에는 100회 이상 구매하지 않은 웹툰은 배제하는걸로 하자.

* 1) 100회 이상 구매한 웹툰만을 색인하기 

In [53]:
comics_purchased = comics[comics["구매 여부"] != 0]
top100 = comics_purchased.groupby("작품 번호").sum()["구매 여부"].sort_values(ascending=False).head(10).index
len(top100)

10

In [55]:
purchased_counts = pd.pivot_table(comics, index="작품 번호", values="구매 여부", aggfunc="sum")
top_purchased_counts = purchased_counts[purchased_counts["구매 여부"] >= 100]

print(top_purchased_counts.shape)
top_purchased_counts.head()

(382, 1)


,구매 여부
작품 번호,
002B4BDE,222
00A49090,5298
01386BD6,1149
0266E33D,135
02E74F10,142


* 2) 이 웹툰의 구매 확률을 계산하기 

In [58]:
top_purchased_index = top_purchased_counts.index

top_comics = comics[comics["작품 번호"].isin(top_purchased_index)]

print(top_comics.shape)
top_comics.head()

(558128, 3)


,구매 여부,작품 번호,구매 코인
4,0,002B4BDE,3
5,0,002B4BDE,3
6,0,00A49090,3
7,0,00A49090,3
8,0,00A49090,3


In [62]:
top_purchased_prob = pd.pivot_table(top_comics, index="작품 번호", values="구매 여부")

print(top_purchased_prob.shape)
top_purchased_prob.head()

(382, 1)


,구매 여부
작품 번호,
002B4BDE,0.382759
00A49090,0.603279
01386BD6,0.413309
0266E33D,0.472028
02E74F10,0.692683


* 3) 마지막으로 확률이 높은 순으로 정렬 후 상위 10개를 출력하기

In [65]:
top_purchased_prob = top_purchased_prob.sort_values(by="구매 여부", ascending=False)

print(top_purchased_prob.shape)
top_purchased_prob.head(10)

(382, 1)


,구매 여부
작품 번호,
149E9677,0.873250
F0935E4C,0.843168
F0A27F22,0.840924
06138BC5,0.825503
BCA82E41,0.806028
3CB79548,0.804795
AD13A2A0,0.801153
10261288,0.799733
D18F655C,0.793467


분석 결과 100개 이상 구매한 작품 중에서는 '149E9677', 'F0935E4C', 'F0A27F22', '06138BC5', 'BCA82E41', '3CB79548', 'AD13A2A0', '10261288', 'D18F655C', '69ADC1E1' 순으로 구매 확률이 가장 높다는 사실을 알 수 있다.

따라서, 위 10개의 작품에 레진코믹스의 사용자 트래픽을 몰아주는게 좋을 것 같다.

## 주요 작품 분석

레진코믹스에서 가장 많이 팔리는 상위 100개의 작품을 골라낸 뒤, 이 작품에 대한 고객의 만족도를 분석하자. 만족도의 기준은, "해당 웹툰을 구매한 뒤 다른 웹툰을 구매했는가?"로 판단하자. 즉, 어떤 웹툰을 굉장히 만족스럽게 보았으면, 사용자는 레진코믹스에 재방문하여 또다른 웹툰을 결제할 것이라고 가정하는 것이다. 반면 어떤 웹툰을 만족스럽게 보지 않았다면, 사용자는 더 이상 레진코믹스에서 웹툰을 구매하지 않을 것이라고 가정할 것이다.

또한 상위 100개의 작품을 기준으로, 웹툰과 웹툰의 연관관계를 분석하자. 가령, "A라는 웹툰과 B라는 웹툰은 서로 연관성이 있기 때문에, A 웹툰을 구매한 사용자에게 B웹툰을 추천하면 구매할 것이다." 라는 연관관게를 찾는 것이 목표이다. 만일 이 관계를 찾을 수 있다면 간단한 컨텐츠 추천엔진을 구현해서 레진코믹스 서비스에 적용할 수 있을 것이다. 이는 아마도 매출 상승이라는 결과를 가져올 것이다.


분석을 위해서는 다음의 컬럼이 필요하다.

  * 1 : label. 해당 유저가 목록에 진입하고 1시간 이내에 구매했는지 여부 (이하 구매 여부)
  * 7 : 작품을 나타내는 해쉬 (이하 작품 번호)
  * 11-110 : 주요 작품 구매 여부. (이하 주요 작품 1 ~ 주요 작품 100)

In [66]:
# 주요 작품 1부터 주요 작품 100까지의 데이터를 가져옵니다.
related_comics = data.loc[:, 10:109].copy()

related_comics.columns = [f"주요 작품 {i+1}" for i in range(related_comics.shape[1])]

print(related_comics.shape)
related_comics.head()

(650965, 100)


,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 91,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100
0,0,11,0,0,1,12,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,25,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0


**11. 가장 많이 구매한 주요 작품 Top 10 찾기**

먼저 주요 작품 100개를 가장 많이 구매한 순으로 정렬하자. 레진코믹스에서 가장 중요한 작품 100개를 추렸기 때문에, 이 데이터에서 정렬 후 상위권으로 나온다면 사실상 레진코믹스에서 가장 중요한 작품이라고 간주할 수 있을 것이다.

In [68]:
# related_comics에 있는 모든 컬럼 값 더하기
related_comics_counts = related_comics.sum()
related_comics_counts = related_comics_counts.sort_values(ascending=False)

related_comics_counts.head(10)

주요 작품 10    2287592
주요 작품 2     2207155
주요 작품 1     1707746
주요 작품 5     1647832
주요 작품 6     1526819
주요 작품 45    1452128
주요 작품 8     1438426
주요 작품 37    1336681
주요 작품 16    1262415
주요 작품 11    1256666
dtype: int64

분석 결과 '주요 작품 10'이 레진코믹스에서 가장 많이 구매된 웹툰이라는 사실을 알 수 있다.

**12. 실제 구매한 사용자들이 가장 많은 횟수로 구매한 주요 작품 Top 10을 찾기**

11번 방식에는 단점이 있는데, 구매하지 않은 사용자(구매 여부 == 0)도 분석에 포함되어 있다는 것이다. 

그러므로 구매하지 않은 사용자는 제거한 후, 가장 많이 팔린 제품 상위 10개를 출력하자.



In [70]:
# related_comices에 구매 여부 컬럼을 추가하기
related_comics["구매 여부"] = data[0]

purchased_related_comics = related_comics[related_comics["구매 여부"] == 1]

purchased_related_comics.head()

,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100,구매 여부
216289,0,34,0,0,12,0,0,0,0,38,...,10,0,0,0,0,0,0,0,0,1
216290,0,10,0,0,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216291,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216292,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216293,0,4,0,0,34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [72]:
purchased_related_comics_counts = purchased_related_comics.sum()
purchased_related_comics_counts = purchased_related_comics_counts.sort_values(ascending=False)

purchased_related_comics_counts.head(10)

주요 작품 10    1326860
주요 작품 2     1211597
주요 작품 5      952626
주요 작품 8      950861
주요 작품 1      931839
주요 작품 45     870640
주요 작품 6      859250
주요 작품 37     845605
주요 작품 3      791907
주요 작품 4      765360
dtype: int64

분석 결과 11의 결과와 같이 '주요 작품 10'이 레진코믹스에서 가장 중요한 웹툰이라는 것을 알 수 있다.

하지만 비구매자를 포함한 결과와는 조금 다르다는 것을 알 수 있는데:
* 주요 작품 1이 순위가 3위에서 5위로 낮아졌다. 
* 주요 작품 5가 4위에서 3위로 높아졌다.
* 주요 작품 8의 순위가 7위에서 4위로 높은 순위를 차지했다는 것을 확인할 수 있다.

**13. 구매 횟수와 상관 없이, 구매 확률이 가장 높은 주요 작품 Top 10 찾기**

이번에는 12번의 연장선상으로, 구매 횟수가 아닌 확률로 계산해보자.

구매 확률은 구매 횟수와 무엇이 다를까? 구매자(구매 여부 == 1)를 기준으로 판단한다면, "사용자가 A 웹툰을 구매하기 전에 B 웹툰을 구매했다면, B웹툰은 사용자가 A웹툰을 구매하는데 긍정적인 영향을 주었을 것이다."라고 판단할 수 있다. 즉, 사용자는 B 웹툰을 본 뒤 큰 만족감을 느낀 뒤 레진코믹스에서 새로운 웹툰을 구매했다고 판단할 수 있는 것이다.

그렇다면 기획팀 입장에서 B웹툰은 자체 매출보다 더 높은 가치를 책정할 수 있다. 가령 B웹툰의 매출이 1억이라고 해도, 실제로는 다른 웹툰의 매출 상승에 기여했기 때문에 1억 이상의 가치가 있다고 판단할 것이다. 회사 입장에서는 이런 저평가된 웹툰을 찾아내는 것은 무엇보다도 중요할 것이다.

이런 저평가된 웹툰을 찾아내기 위해, 구매 확률이 가장 높은 웹툰을 찾아보자. 여기서 주의할 점은, 사용자가 특정 웹툰을 구매를 20번 하건 30번 하건 전부 1번으로 간주해야 한다는 것이다. 가령 A라는 웹툰은 전체 10편인데 10편을 모두 결제했고, B라는 웹툰은 전체 100편인데 20편만 결제했다면, 구매 횟수 상으로는 B가 높지만(10회 < 20회) 확률은 A가 높다(100% > 20%). 하지만 이 데이터에서는 각 웹툰이 총 몇 편인지 알 수 없기 때문에 사용자가 웹툰을 '구매했다(1)/구매하지 않았다(0)'로만 나눌 것이다.

즉, 전체 650,965 개의 데이터에서 주요 작품 1번을 6,509번 구매했다면 구매 확률은 0.01(1%)로 가정한다.

In [78]:
# purchased_related_comics에 있는 모든 값을 True/False로 바꿉니다.
purchased_related_comics_binary = purchased_related_comics.astype('bool').astype('int')
purchased_related_comics_binary = purchased_related_comics_binary.drop("구매 여부", axis=1)

print(purchased_related_comics_binary.shape)
purchased_related_comics_binary.head()

(221793, 100)


,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 91,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100
216289,0,1,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
216290,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216291,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216292,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
216293,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
purchased_related_comics_binary_means = purchased_related_comics_binary.mean()
purchased_related_comics_binary_means = purchased_related_comics_binary_means.sort_values(ascending=False)

purchased_related_comics_binary_means.head(10)

주요 작품 2     0.307426
주요 작품 18    0.258953
주요 작품 10    0.258326
주요 작품 11    0.229092
주요 작품 5     0.228812
주요 작품 13    0.227013
주요 작품 41    0.206156
주요 작품 52    0.199582
주요 작품 39    0.197770
주요 작품 6     0.195773
dtype: float64

분석 결과, 주요 작품 2의 구매 확률이 30%로 매우 높은 편이다. 즉, 레진코믹스에서 웹툰을 결제한 사람은 30%의 확률로 주요 작품 2를 한 번 이상 구매한 사람이라고 판단할 수 있다. 그 다음은 주요 작품 18번의 순위가 높습니다(구매량을 기준으로 정렬했을때는 등장하지 않은 작품명). 

따라서 주요 작품 2에 대한 사용자 트래픽을 더 높이는 쪽으로 방향을 잡으면 매출에 좋을 것이다.

**14. 작품 번호별 주요 작품의 구매 회수를 구하기**

이번에는 현재 작품 번호를 기준으로, 이전에 구매한 주요 작품의 구매 회수를 알고 싶다.

이 데이터를 보유하고 있으면, 우리는 간단한 웹툰 추천 엔진을 만들 수 있다. (가령 주요 작품 2번을 구매한 사람에게는 구매 수가 가장 높았던D619AC7D 작품을 추천해줄 수 있다.) 만일 서비스 내에 추천 엔진이 들어가 있지 않거나, 그 성능이 크게 좋지 않다면 이 정보를 바탕으로 구현한 웹툰 추천 엔진을 사용할 수 있을 것입니다.

In [80]:
related_comics_binary = purchased_related_comics.astype('bool').astype('int')
related_comics_binary["구매 여부"] = data[0]
related_comics_binary["작품 번호"] = data[6]

purchased_related_comics_binary = related_comics_binary[related_comics_binary["구매 여부"] == 1]

print(purchased_related_comics_binary.shape)
purchased_related_comics_binary.head()

(221793, 102)


,주요 작품 1,주요 작품 2,주요 작품 3,주요 작품 4,주요 작품 5,주요 작품 6,주요 작품 7,주요 작품 8,주요 작품 9,주요 작품 10,...,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99,주요 작품 100,구매 여부,작품 번호
216289,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,001C9D9B
216290,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,002B4BDE
216291,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,00A49090
216292,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,00A49090
216293,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,00A49090


In [82]:
table = pd.pivot_table(related_comics_binary,
                       index="작품 번호",
                       values=[f"주요 작품 {i+1}" for i in range(100)],
                       aggfunc=np.sum)

table.head()

,주요 작품 1,주요 작품 10,주요 작품 100,주요 작품 11,주요 작품 12,주요 작품 13,주요 작품 14,주요 작품 15,주요 작품 16,주요 작품 17,...,주요 작품 90,주요 작품 91,주요 작품 92,주요 작품 93,주요 작품 94,주요 작품 95,주요 작품 96,주요 작품 97,주요 작품 98,주요 작품 99
작품 번호,,,,,,,,,,,,,,,,,,,,,
001C9D9B,21,71,34,33,7,62,1,11,45,8,...,1,2,47,5,26,3,1,1,6,6
002B4BDE,70,145,57,76,14,131,8,47,89,18,...,1,4,111,9,64,7,4,0,16,15
0030E221,2,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,0,0,0,0,0
00343701,2,2,2,2,0,2,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
00545B4F,7,21,8,13,3,15,1,9,19,0,...,0,0,17,3,6,1,1,0,2,2


**15. 14번 데이터를 1) 가장 많이 구매한 작품 번호 2) 가장 많이 구매한 주요 작품 순으로 정렬하기**

14번 데이터는 경우의 수가 너무 많아 (3559x100) 눈으로 분석하기 매우 불편하다. 

따라서 데이터의 row와 column을 다음과 같이 정렬하자. 

먼저 가장 많이 구매한(누적 구매량) 작품 번호가 표의 상단으로 오도록 정렬한 후, 가장 많이 구매한(누적 구매량) 주요 작품이 표의 좌측으로 오도록 정렬하자. 

In [83]:
index = table.sum(axis=1).sort_values(ascending=False).index
columns = table.sum(axis=0).sort_values(ascending=False).index

table.loc[index, columns].head(10)

,주요 작품 2,주요 작품 18,주요 작품 10,주요 작품 11,주요 작품 5,주요 작품 13,주요 작품 41,주요 작품 52,주요 작품 39,주요 작품 6,...,주요 작품 17,주요 작품 82,주요 작품 91,주요 작품 60,주요 작품 65,주요 작품 93,주요 작품 40,주요 작품 90,주요 작품 50,주요 작품 97
작품 번호,,,,,,,,,,,,,,,,,,,,,
F0A27F22,838,1508,1399,1303,404,339,1887,2399,1186,378,...,311,157,328,605,189,215,218,114,56,51
D619AC7D,3109,5684,3178,1703,1503,1668,848,1607,2627,1895,...,495,168,122,206,41,38,100,92,5,90
EC612BAF,764,1523,1537,1117,500,422,1454,2050,1282,475,...,522,335,340,919,189,135,396,180,118,68
BEE0E409,867,1136,1022,953,359,346,1211,3298,866,444,...,155,114,208,265,132,94,103,116,18,44
3942C7FE,1404,1700,1710,656,810,801,433,807,1451,1010,...,335,104,47,111,14,43,58,44,2,36
707F7107,1910,1569,1561,943,1478,1408,433,435,1171,1465,...,160,62,40,11,13,34,33,51,2,24
85803B60,3891,1170,1381,998,1316,1271,393,591,815,1153,...,146,80,34,32,22,56,33,45,1,22
00A49090,1743,822,1048,611,5284,2127,267,270,565,1095,...,135,79,26,22,17,28,98,56,18,10
B55E3754,441,540,507,492,186,163,930,938,405,216,...,78,53,132,150,102,102,83,48,17,14


15번에서 출력한 데이터가 14번에서 출력한 데이터보다 더 분석하기 편한 것 같다.